# Setup

In [ ]:
! pip install ONE-api
! pip install ibllib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.3/996.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.6/111.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.0/206.0 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.4/80.4 kB 6.1 MB/s eta 0

In [ ]:
# Standard libraries
from pprint import pprint
import random
from typing import List, Dict, Optional, Tuple, Union
import warnings
import os
import requests
from pathlib import Path
import time

# Third-party libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.gridspec import GridSpec
import matplotlib as mpl
from matplotlib.lines import Line2D
from scipy.ndimage import uniform_filter1d
from IPython.display import HTML, display

# IBL-specific libraries
from iblatlas.atlas import BrainAtlas
from iblatlas import atlas
from ibllib.io import video
from one.api import ONE, OneAlyx
from brainbox.io.one import SessionLoader

from one.api import ONE
from brainbox.io.one import SessionLoader
import ibllib.io.video as video
from iblutil.util import Bunch

# Matplotlib animation limit
mpl.rcParams['animation.embed_limit'] = 200

# Google Drive
from google.colab import drive
drive.mount('/content/drive')
BASE_DIR = "content/drive/My Drive/S25/Langone/Breathing/Videos" # capital Drive is for cloud, drive is for local

Mounted at /content/drive


In [ ]:
ONE.setup(base_url='https://openalyx.internationalbrainlab.org', silent=True)
one = ONE(password='international')

Connected to https://openalyx.internationalbrainlab.org as user "intbrainlab"


# IBLDataLoader

In [ ]:
import numpy as np
import pandas as pd
from one.api import ONE
from brainbox.io.one import SessionLoader
import ibllib.io.video as video
from typing import Dict, Optional

class IBLDataLoader:
    """
    Loads and organizes IBL behavioral data for animation purposes.

    This class handles session loading, data validation, interval sampling,
    and synchronization between video timestamps and behavioral features.
    """

    def __init__(self, eid: str, verbose: bool = True):
        """
        Initialize the data loader for a specific experiment.

        Args:
            eid (str): Experiment ID to load
            verbose (bool): Enable detailed logging and debugging output
        """
        self.eid = eid
        self.verbose = verbose
        self.one = ONE()
        self.sessLoader = None
        self.sessionInfo = {}
        self.trialData = None
        self.poseData = None
        self.wheelData = None
        self.motionEnergyData = None
        self.pupilData = None
        self.fps = None # dictionary left, right, body
        self.videoTimes = None # Default aligns time to left camera, can be changed in loadVideoTimestamps() self.videoTimes = timestamps['left']
        self.urls = None

        # Features we want to track from pose data
        self.requiredPoseFeatures = [
            'times', 'nose_tip_x', 'nose_tip_y', 'pupil_top_r_x', 'pupil_top_r_y',
            'pupil_right_r_x', 'pupil_right_r_y', 'pupil_bottom_r_x', 'pupil_bottom_r_y',
            'pupil_left_r_x', 'pupil_left_r_y', 'tube_top_x', 'tube_top_y',
            'tube_bottom_x', 'tube_bottom_y', 'tongue_end_l_x', 'tongue_end_l_y',
            'tongue_end_r_x', 'tongue_end_r_y'
        ]

        # Trial event features we want to extract
        self.requiredTrialFeatures = [
            'intervals_0', 'intervals_1', 'stimOff_times', 'goCueTrigger_times',
            'stimOn_times', 'response_times', 'goCue_times', 'firstMovement_times',
            'feedback_times'
        ]

        if self.verbose:
            print(f"Initializing IBL DataLoader for experiment: {eid}")

    def loadSessionData(self) -> bool:
        """
        Load all required session data using SessionLoader.

        Returns:
            bool: True if loading successful, False otherwise
        """
        try:
            if self.verbose:
                print("Loading session data...")

            # Initialize SessionLoader
            self.sessLoader = SessionLoader(one=self.one, eid=self.eid)

            # Load all required data types
            self.sessLoader.load_session_data(
                trials=True,           # Task timing events
                wheel=True,            # Wheel movement data
                pose=True,             # Pose tracking (all cameras)
                motion_energy=True,    # Motion energy (all cameras)
                pupil=True,            # Pupil diameter measurements
                reload=False
            )

            if self.verbose:
                print("✓ Session data loaded successfully")

            return True

        except Exception as e:
            print(f"✗ Error loading session data: {str(e)}")
            return False

    def extractSessionInfo(self) -> Dict:
        """
        Extract comprehensive session metadata with robust path handling.

        Returns:
            Dict: Session information including subject, date, lab, etc.
        """
        try:
            if self.verbose:
                print("Extracting session information...")

            sessionDetails = self.one.get_details(self.eid)

            # Handle different path object types that ONE API might return
            sessionPath = sessionDetails.get('local_path', 'Unknown')

            # Convert path object to string if needed
            if hasattr(sessionPath, '__str__'):
                sessionPathStr = str(sessionPath)
            elif hasattr(sessionPath, 'as_posix'):
                sessionPathStr = sessionPath.as_posix()
            else:
                sessionPathStr = sessionPath

            if self.verbose:
                print(f"  Raw session path: {sessionPathStr}")

            # Parse path components for session metadata with defensive splitting
            pathParts = sessionPathStr.replace('\\', '/').split('/')  # Handle both Unix and Windows paths
            pathParts = [part for part in pathParts if part]  # Remove empty parts

            # Extract session components with fallback logic
            if len(pathParts) >= 3:
                subject = pathParts[-3]
                date = pathParts[-2]
                sessionNum = pathParts[-1]
            elif len(pathParts) >= 2:
                subject = pathParts[-2]
                date = pathParts[-1]
                sessionNum = 'Unknown'
            elif len(pathParts) >= 1:
                subject = pathParts[-1]
                date = 'Unknown'
                sessionNum = 'Unknown'
            else:
                subject = 'Unknown'
                date = 'Unknown'
                sessionNum = 'Unknown'

            # Alternative extraction method using session details if path parsing fails
            if subject == 'Unknown' and 'subject' in sessionDetails:
                subject = sessionDetails['subject']
            if date == 'Unknown' and 'start_time' in sessionDetails:
                # Extract date from start_time if available
                startTime = sessionDetails['start_time']
                if hasattr(startTime, 'date'):
                    date = startTime.date().isoformat()
                elif isinstance(startTime, str) and len(startTime) >= 10:
                    date = startTime[:10]  # Extract YYYY-MM-DD portion

            # Get accurate trial count using SessionLoader
            try:
                if self.sessLoader is None:
                    temp_sess_loader = SessionLoader(one=self.one, eid=self.eid)
                    temp_sess_loader.load_trials()
                    numberOfTrials = temp_sess_loader.trials.shape[0]
                else:
                    # Use existing sessLoader if available
                    if hasattr(self.sessLoader, 'trials') and self.sessLoader.trials is not None:
                        numberOfTrials = self.sessLoader.trials.shape[0]
                    else:
                        self.sessLoader.load_trials()
                        numberOfTrials = self.sessLoader.trials.shape[0]
            except Exception as e:
                if self.verbose:
                    print(f"  Warning: Could not load trial count via SessionLoader: {e}")
                numberOfTrials = sessionDetails.get('n_trials', 'Unknown')

            self.sessionInfo = {
                'experimentId': self.eid,
                'subject': subject,
                'date': date,
                'sessionNumber': sessionNum,
                'lab': sessionDetails.get('lab', 'Unknown'),
                'taskProtocol': sessionDetails.get('task_protocol', 'Unknown'),
                'numberOfTrials': numberOfTrials,
                'sessionPath': sessionPathStr,
                'dataRepository': sessionDetails.get('data_repository', 'Unknown')
            }

            if self.verbose:
                print("✓ Session info extracted:")
                for key, value in self.sessionInfo.items():
                    print(f"  {key}: {value}")

            return self.sessionInfo

        except Exception as e:
            print(f"✗ Error extracting session info: {str(e)}")
            if self.verbose:
                print(f"  Session details keys available: {list(sessionDetails.keys()) if 'sessionDetails' in locals() else 'None'}")

            # Return minimal info with experiment ID even if extraction fails
            self.sessionInfo = {
                'experimentId': self.eid,
                'subject': 'Unknown',
                'date': 'Unknown',
                'sessionNumber': 'Unknown',
                'lab': 'Unknown',
                'taskProtocol': 'Unknown',
                'numberOfTrials': 'Unknown',
                'sessionPath': 'Unknown',
                'dataRepository': 'Unknown'
            }
            return self.sessionInfo

    def validateAndExtractTrialData(self) -> bool:
        """
        Validate and extract trial data with comprehensive feature checking.

        Returns:
            bool: True if validation successful, False otherwise
        """
        try:
            if self.verbose:
                print("Validating and extracting trial data...")

            if not hasattr(self.sessLoader, 'trials') or self.sessLoader.trials is None:
                print("✗ No trial data available")
                return False

            self.trialData = self.sessLoader.trials

            # Check for required trial features
            availableFeatures = list(self.trialData.columns)
            missingFeatures = [f for f in self.requiredTrialFeatures if f not in availableFeatures]

            if self.verbose:
                print(f"✓ Trial data loaded: {len(self.trialData)} trials")
                print(f"  Available features: {len(availableFeatures)}")
                if missingFeatures:
                    print(f"  Missing features: {missingFeatures}")
                else:
                    print("  ✓ All required trial features present")

                # Show trial interval statistics
                if 'intervals_0' in availableFeatures and 'intervals_1' in availableFeatures:
                    intervalDurations = self.trialData['intervals_1'] - self.trialData['intervals_0']
                    print(f"  Trial duration stats:")
                    print(f"    Mean: {intervalDurations.mean():.2f}s")
                    print(f"    Min: {intervalDurations.min():.2f}s")
                    print(f"    Max: {intervalDurations.max():.2f}s")

            return len(missingFeatures) == 0

        except Exception as e:
            print(f"✗ Error validating trial data: {str(e)}")
            return False

    def validateAndExtractPoseData(self) -> bool:
        """
        Validate and extract left camera pose tracking data.

        Returns:
            bool: True if validation successful, False otherwise
        """
        try:
            if self.verbose:
                print("Validating and extracting pose data...")

            if not hasattr(self.sessLoader, 'pose') or 'leftCamera' not in self.sessLoader.pose:
                print("✗ No left camera pose data available")
                return False

            self.poseData = self.sessLoader.pose['leftCamera']

            # Check for required pose features
            availableFeatures = list(self.poseData.columns)
            missingFeatures = [f for f in self.requiredPoseFeatures if f not in availableFeatures]

            if self.verbose:
                print(f"✓ Pose data loaded: {len(self.poseData)} timepoints")
                print(f"  Available features: {len(availableFeatures)}")
                if missingFeatures:
                    print(f"  Missing features: {missingFeatures}")
                else:
                    print("  ✓ All required pose features present")

                # Show data completeness for each feature
                print("  Data completeness check:")
                for feature in self.requiredPoseFeatures:
                    if feature in availableFeatures:
                        nonNullCount = self.poseData[feature].notna().sum()
                        totalCount = len(self.poseData)
                        completeness = (nonNullCount / totalCount) * 100
                        print(f"    {feature}: {nonNullCount}/{totalCount} ({completeness:.1f}%)")

                # Show temporal coverage
                if 'times' in availableFeatures:
                    timeRange = self.poseData['times'].max() - self.poseData['times'].min()
                    print(f"  Temporal coverage: {timeRange:.2f} seconds")

            return len(missingFeatures) == 0

        except Exception as e:
            print(f"✗ Error validating pose data: {str(e)}")
            return False

    def extractOtherBehavioralData(self) -> bool:
        """
        Extract wheel, motion energy, and pupil data using SessionLoader methods.
        Reports what features are tracked across all cameras (left, right, body).

        Returns:
            bool: True if extraction successful, False otherwise
        """
        try:
            if self.verbose:
                print("Extracting additional behavioral data...")

            # Load wheel data
            try:
                self.sessLoader.load_wheel()
                if hasattr(self.sessLoader, 'wheel') and self.sessLoader.wheel is not None:
                    self.wheelData = self.sessLoader.wheel
                    if self.verbose:
                        print(f"✓ Wheel data: {len(self.wheelData)} timepoints")
                else:
                    if self.verbose:
                        print("! No wheel data available")
            except Exception as e:
                if self.verbose:
                    print(f"! Failed to load wheel data: {e}")

            # Load pose data for all cameras (left, right, body)
            try:
                self.sessLoader.load_pose(views=['left', 'right', 'body'])
                if hasattr(self.sessLoader, 'pose'):
                    cameras = ['leftCamera', 'rightCamera', 'bodyCamera']
                    available_cameras = [cam for cam in cameras if cam in self.sessLoader.pose]

                    if self.verbose:
                        print(f"✓ Pose data loaded for cameras: {available_cameras}")

                    # Check what features are tracked across cameras
                    if available_cameras:
                        # Get features from each camera
                        camera_features = {}
                        for cam in available_cameras:
                            if self.sessLoader.pose[cam] is not None:
                                camera_features[cam] = set(self.sessLoader.pose[cam].columns)

                        if camera_features and self.verbose:
                            # Find common features across all cameras
                            common_features = set.intersection(*camera_features.values()) if len(camera_features) > 1 else next(iter(camera_features.values()))

                            print(f"  Feature analysis across {len(camera_features)} cameras:")
                            print(f"    Common features ({len(common_features)}): {sorted(common_features)}")

                            for cam, features in camera_features.items():
                                unique_features = features - common_features
                                if unique_features:
                                    print(f"    {cam} unique features ({len(unique_features)}): {sorted(unique_features)}")

                            # Report on pupil features specifically
                            pupil_features = {cam: [f for f in features if f.startswith('pupil_')]
                                            for cam, features in camera_features.items()}
                            pupil_available = {cam: features for cam, features in pupil_features.items() if features}

                            if pupil_available:
                                print(f"    Pupil tracking available in: {list(pupil_available.keys())}")
                                for cam, features in pupil_available.items():
                                    print(f"      {cam}: {features}")

                        # Set primary data sources
                        if 'leftCamera' in self.sessLoader.pose:
                            self.poseData = self.sessLoader.pose['leftCamera']
                            self.pupilData = self.sessLoader.pose['leftCamera']  # Pupil data is part of pose

                else:
                    if self.verbose:
                        print("! No pose data available")
            except Exception as e:
                if self.verbose:
                    print(f"! Failed to load pose data: {e}")

            # Load motion energy data for all cameras
            try:
                self.sessLoader.load_motion_energy(views=['left', 'right', 'body'])
                if hasattr(self.sessLoader, 'motion_energy'):
                    cameras = ['leftCamera', 'rightCamera', 'bodyCamera']
                    available_cameras = [cam for cam in cameras if cam in self.sessLoader.motion_energy]

                    if self.verbose:
                        print(f"✓ Motion energy data loaded for cameras: {available_cameras}")

                    # Check what motion features are available
                    if available_cameras:
                        for cam in available_cameras:
                            motion_data = self.sessLoader.motion_energy[cam]
                            if isinstance(motion_data, dict):
                                features = list(motion_data.keys())
                            else:
                                features = ['whiskerMotionEnergy']

                            if self.verbose:
                                print(f"    {cam}: {features}")

                        # Set primary motion energy source
                        if 'leftCamera' in self.sessLoader.motion_energy:
                            self.motionEnergyData = self.sessLoader.motion_energy['leftCamera']
                else:
                    if self.verbose:
                        print("! No motion energy data available")
            except Exception as e:
                if self.verbose:
                    print(f"! Failed to load motion energy data: {e}")

            # Load pupil diameter data (separate from pose)
            try:
                self.sessLoader.load_pupil()
                if hasattr(self.sessLoader, 'pupil') and self.sessLoader.pupil is not None:
                    cameras = ['leftCamera', 'rightCamera', 'bodyCamera']
                    available_cameras = [cam for cam in cameras if cam in self.sessLoader.pupil]

                    if available_cameras and self.verbose:
                        print(f"✓ Dedicated pupil data loaded for cameras: {available_cameras}")
                else:
                    if self.verbose:
                        print("! No dedicated pupil data (pupil tracking is in pose data)")
            except Exception as e:
                if self.verbose:
                    print(f"! Failed to load dedicated pupil data: {e}")

            return True

        except Exception as e:
            print(f"✗ Error extracting behavioral data: {str(e)}")
            return False

    def loadVideoTimestamps(self) -> bool:
        """
        Load video timestamps for left, body, and right cameras.

        Performs two key operations:
        1. Aligns camera times to trial intervals (intervals_0 to intervals_1)
        2. Standardizes all cameras to 24 fps using frame decimation/upsampling:
           - For cameras > 24fps (e.g., 120fps): Take every Nth frame (120fps → take every 5th frame)
           - For cameras < 24fps (e.g., 30fps): Use frame repetition/interpolation to upsample
           - For cameras = 24fps: Use frames as-is

        Frame decimation examples:
        - 30fps → 24fps: Keep 4 out of every 5 frames (30/24 = 1.25, so skip every 5th frame)
        - 60fps → 24fps: Keep 2 out of every 5 frames (60/24 = 2.5, so take every 2.5th frame)
        - 120fps → 24fps: Keep 1 out of every 5 frames (120/24 = 5, so take every 5th frame)

        Returns:
            bool: True if loading successful, False otherwise
        """
        try:
            if self.verbose:
                print("Loading video timestamps...")

            eid = self.eid
            one = self.one
            labels = ['left', 'right', 'body']

            # Get all video URLs at once for efficiency
            video_urls = video.url_from_eid(eid, one=one)
            urls = {label: video_urls[label] for label in labels}

            # Load timestamps for each camera
            timestamps = {}
            for label in labels:
                timestamps[label] = one.load_dataset(eid, f'*{label}Camera.times*', collection='alf')

            # Report on video frame counts (they may differ)
            if self.verbose:
                frame_counts = {label: len(timestamps[label]) for label in labels}
                print(f"Video frame counts: {frame_counts}")
                if len(set(frame_counts.values())) == 1:
                    print("✓ All cameras have the same number of frames")
                else:
                    print("! Cameras have different numbers of frames (this is normal)")

            self.urls = urls
            self.videoTimes = timestamps['left'] # default assigns video times to left camera

            # Get actual video durations and calculate original FPS for each camera
            original_fps = {}
            video_durations = {}

            for label in labels:
                try:
                    # Get actual video metadata for duration
                    meta = video.get_video_meta(urls[label], one=one)
                    actual_duration = meta.get('duration')

                    # Handle datetime.timedelta objects
                    if hasattr(actual_duration, 'total_seconds'):
                        actual_duration = actual_duration.total_seconds()

                    video_durations[label] = actual_duration

                    # Calculate FPS using actual video duration
                    num_frames = len(timestamps[label])
                    if actual_duration and actual_duration > 0:
                        calculated_fps = num_frames / actual_duration
                        original_fps[label] = round(calculated_fps)  # Round to nearest integer
                        if self.verbose:
                            print(f"  {label}: {num_frames} frames / {actual_duration:.2f}s = {calculated_fps:.2f} fps → rounded to {original_fps[label]} fps")
                    else:
                        # Fallback to timestamp range if metadata fails
                        timestamp_duration = timestamps[label][-1] - timestamps[label][0]
                        calculated_fps = num_frames / timestamp_duration
                        original_fps[label] = round(calculated_fps)
                        video_durations[label] = timestamp_duration
                        if self.verbose:
                            print(f"  {label}: Using timestamp duration fallback - {calculated_fps:.2f} fps → rounded to {original_fps[label]} fps")

                except Exception as e:
                    # Fallback to timestamp-based calculation
                    timestamp_duration = timestamps[label][-1] - timestamps[label][0]
                    num_frames = len(timestamps[label])
                    calculated_fps = num_frames / timestamp_duration
                    original_fps[label] = round(calculated_fps)
                    video_durations[label] = timestamp_duration
                    if self.verbose:
                        print(f"  {label}: Metadata failed ({e}), using timestamp fallback - {calculated_fps:.2f} fps → rounded to {original_fps[label]} fps")

            # CORRECTED APPROACH: Preserve original camera timing instead of forcing standardization
            if self.verbose:
                print(f"\nPreserving original camera timing (NO frame rate standardization):")
                print("  This maintains natural timing relationships between cameras")
                for label in labels:
                    original_times = timestamps[label]
                    original_camera_fps = original_fps[label]
                    print(f"  {label}: {len(original_times)} frames at {original_camera_fps:.1f} fps (PRESERVED)")

            # Store results with original timing preserved
            self.fps = {
                'original': original_fps,
                'preserved': original_fps,  # Same as original - no standardization
                'target': None  # No target fps - respect original rates
            }

            # Use left camera as master timeline but preserve all original timestamps
            self.videoTimes = timestamps['left']  # Master timeline for animation
            self.originalTimestamps = timestamps  # Store ALL original timestamps

            # Remove the problematic standardizedTimestamps that destroy timing
            # Instead, provide raw access to original timestamps for each camera

            if self.verbose:
                print(f"\n✓ Video processing completed:")
                print(f"  Original timestamps preserved for all cameras")
                print(f"  No frame rate standardization applied")
                print(f"  Primary video time base: left camera ({len(self.videoTimes)} frames)")
                print(f"✓ URLs loaded for all cameras")

            return True
        except Exception as e:
            print(f"✗ Error loading video timestamps and or urls: {str(e)}")
            return False

    def sampleInterval(self, trialIdx: int) -> Optional[Dict]:
        """
        Sample a specific trial interval from the available trials.

        Returns:
            Dict: Sampled interval data with trial info, or None if sampling fails
        """
        try:
            if self.trialData is None or len(self.trialData) == 0:
                print("✗ No trial data available for sampling")
                return None
            print(f"There are {len(self.trialData)} trials available")
            print(f"Sampling trial {trialIdx}")

            selectedTrial = self.trialData.iloc[trialIdx]
            intervalStart = selectedTrial['intervals_0']
            intervalEnd = selectedTrial['intervals_1']
            intervalDuration = intervalEnd - intervalStart

            if self.verbose:
                print(f" Successfully sampled trial {trialIdx}, Interval: {intervalStart:.3f} - {intervalEnd:.3f}s")

            # Extract synchronized pose data for this interval
            poseInInterval = self.extractPoseDataForInterval(intervalStart, intervalEnd)
            intervalData = {
                'trialIndex': trialIdx,
                'intervalStart': intervalStart,
                'intervalEnd': intervalEnd,
                'duration': intervalDuration,
                'trialEvents': selectedTrial.to_dict(),
                'poseData': poseInInterval,
                'sessionInfo': self.sessionInfo
            }

            return intervalData
        except Exception as e:
            print(f"Error sampling interval: {str(e)}")
            return None

    def extractPoseDataForInterval(self, startTime: float, endTime: float) -> Optional[pd.DataFrame]:
        """
        Extract pose data that falls within the specified time interval.
        Handles missing data by preserving original timestamps for linear interpolation.

        Args:
            startTime (float): Start time of interval in seconds
            endTime (float): End time of interval in seconds

        Returns:
            pd.DataFrame: Pose data within the time interval, or None if extraction fails
        """
        try:
            if self.poseData is None or 'times' not in self.poseData.columns:
                print("✗ No pose data or timestamps available")
                return None

            # Find pose data within the time interval
            timeFilter = (self.poseData['times'] >= startTime) & (self.poseData['times'] <= endTime)
            intervalPoseData = self.poseData[timeFilter].copy()

            if len(intervalPoseData) == 0:
                print("! No pose data in specified interval")
                return None

            # Calculate data availability statistics for this interval
            if self.verbose:
                print(f"  Pose data in interval: {len(intervalPoseData)} timepoints")

                # Show feature availability for this specific interval
                featureStats = {}
                for feature in self.requiredPoseFeatures:
                    if feature in intervalPoseData.columns and feature != 'times':
                        validCount = intervalPoseData[feature].notna().sum()
                        totalCount = len(intervalPoseData)
                        completeness = (validCount / totalCount) * 100
                        featureStats[feature] = {
                            'valid_points': validCount,
                            'total_points': totalCount,
                            'completeness': completeness
                        }

                # Show summary of feature availability
                highQualityFeatures = [f for f, stats in featureStats.items() if stats['completeness'] > 80]
                mediumQualityFeatures = [f for f, stats in featureStats.items() if 20 <= stats['completeness'] <= 80]
                lowQualityFeatures = [f for f, stats in featureStats.items() if stats['completeness'] < 20]

                print(f"    High quality features (>80%): {len(highQualityFeatures)}")
                print(f"    Medium quality features (20-80%): {len(mediumQualityFeatures)}")
                print(f"    Low quality features (<20%): {len(lowQualityFeatures)}")

                if lowQualityFeatures and self.verbose:
                    print(f"    Low quality features will rely on interpolation: {lowQualityFeatures[:3]}{'...' if len(lowQualityFeatures) > 3 else ''}")

            return intervalPoseData

        except Exception as e:
            print(f"✗ Error extracting pose data for interval: {str(e)}")
            return None

    def extractWheelDataForInterval(self, startTime: float, endTime: float) -> Optional[Dict]:
        """
        Extract wheel data that falls within the specified time interval.
        Uses sess_loader.wheel which has columns: ['times', 'position', 'velocity', 'acceleration'].

        Args:
            startTime (float): Start time of interval in seconds
            endTime (float): End time of interval in seconds

        Returns:
            Dict: Wheel data within the time interval with times and position_raw, or None if extraction fails
        """
        try:
            if self.wheelData is None:
                if self.verbose:
                    print("✗ No wheel data available")
                return None

            # Check if wheelData has required columns
            if not hasattr(self.wheelData, 'columns') or 'times' not in self.wheelData.columns or 'position' not in self.wheelData.columns:
                if self.verbose:
                    print("✗ Wheel data missing required columns (times, position)")
                return None

            # Filter wheel data within the time interval
            timeFilter = (self.wheelData['times'] >= startTime) & (self.wheelData['times'] <= endTime)
            intervalWheelData = self.wheelData[timeFilter].copy()

            if len(intervalWheelData) == 0:
                if self.verbose:
                    print("! No wheel data in specified interval")
                return None

            # Return wheel data in the format expected by AnimationRenderer
            wheelData = {
                'times': intervalWheelData['times'].values,
                'position_raw': intervalWheelData['position'].values,
            }

            if self.verbose:
                print(f"✓ Wheel data extracted: {len(intervalWheelData)} timepoints")
                print(f"  Time range: {wheelData['times'][0]:.3f} - {wheelData['times'][-1]:.3f}s")
                print(f"  Position range: {np.nanmin(wheelData['position_raw']):.6f} - {np.nanmax(wheelData['position_raw']):.6f}")
                print(f"  Valid positions: {np.sum(~np.isnan(wheelData['position_raw']))}")

            return wheelData

        except Exception as e:
            if self.verbose:
                print(f"✗ Error extracting wheel data for interval: {str(e)}")
            return None

    def extractMotionEnergyForInterval(self, startTime: float, endTime: float) -> Optional[Dict]:
        """
        Extract motion energy data that falls within the specified time interval.
        Uses sess_loader.motion_energy['leftCamera'] structure as user indicated.

        Args:
            startTime (float): Start time of interval in seconds
            endTime (float): End time of interval in seconds

        Returns:
            Dict: Motion energy data within the time interval, or None if extraction fails
        """
        try:
            if self.motionEnergyData is None:
                if self.verbose:
                    print("✗ No motion energy data available")
                return None

            if self.verbose:
                print(f"Motion energy data type: {type(self.motionEnergyData)}")
                if hasattr(self.motionEnergyData, 'columns'):
                    print(f"Motion energy DataFrame columns: {list(self.motionEnergyData.columns)}")
                elif isinstance(self.motionEnergyData, dict):
                    print(f"Motion energy keys: {list(self.motionEnergyData.keys())}")

            # Extract whisker motion energy and timestamps
            # Based on user info: sess_loader.motion_energy['leftCamera'].columns
            whiskerData = None
            timestamps = None

            # Case 1: DataFrame with columns (most likely structure)
            if hasattr(self.motionEnergyData, 'columns'):
                df = self.motionEnergyData

                # Look for whisker motion energy column
                whisker_col = None
                for col in df.columns:
                    col_str = str(col).lower()
                    if 'whisker' in col_str and ('motion' in col_str or 'energy' in col_str):
                        whisker_col = col
                        break
                    elif 'motionenergy' in col_str.replace('_', '').replace(' ', ''):
                        whisker_col = col
                        break

                if whisker_col is None and len(df.columns) > 0:
                    # Use first column as fallback
                    whisker_col = df.columns[0]
                    if self.verbose:
                        print(f"No clear whisker column found, using first column: '{whisker_col}'")

                if whisker_col is not None:
                    whiskerData = df[whisker_col]
                    if self.verbose:
                        print(f"Using column '{whisker_col}' for whisker motion energy")

                # Get timestamps - check for 'times' column or use index
                if 'times' in df.columns:
                    timestamps = df['times']
                elif hasattr(df.index, 'values'):
                    timestamps = df.index.values
                else:
                    timestamps = df.index

            # Case 2: Dictionary structure
            elif isinstance(self.motionEnergyData, dict):
                # Look for whisker motion energy in dictionary
                possible_keys = ['whiskerMotionEnergy', 'whisker', 'motion_energy', 'leftCamera']
                for key in possible_keys:
                    if key in self.motionEnergyData:
                        whiskerData = self.motionEnergyData[key]
                        break

                # Look for timestamps
                if 'times' in self.motionEnergyData:
                    timestamps = self.motionEnergyData['times']
                elif hasattr(whiskerData, 'index'):
                    timestamps = whiskerData.index

            # Case 3: Direct Series/Array
            else:
                whiskerData = self.motionEnergyData
                if hasattr(whiskerData, 'index'):
                    timestamps = whiskerData.index

            if whiskerData is None:
                if self.verbose:
                    print("✗ No whisker motion energy data found")
                return None

            if timestamps is None:
                if self.verbose:
                    print("✗ No timestamps found for motion energy data")
                return None

            # Convert to numpy arrays for filtering
            if hasattr(timestamps, 'values'):
                timestamps_array = timestamps.values
            else:
                timestamps_array = np.asarray(timestamps)

            if hasattr(whiskerData, 'values'):
                whisker_array = whiskerData.values
            else:
                whisker_array = np.asarray(whiskerData)

            # Filter data for time interval
            timeFilter = (timestamps_array >= startTime) & (timestamps_array <= endTime)
            intervalTimestamps = timestamps_array[timeFilter]
            intervalWhiskerData = whisker_array[timeFilter]

            if len(intervalTimestamps) == 0:
                if self.verbose:
                    print("! No motion energy data in specified interval")
                return None

            # Ensure whisker data is 1D
            intervalWhiskerData = np.asarray(intervalWhiskerData).flatten()

            # Apply smoothing
            smoothingWindow = 0.05
            if len(intervalTimestamps) > 1:
                estimatedSamplingRate = 1.0 / np.median(np.diff(intervalTimestamps))
                windowSamples = max(1, int(smoothingWindow * estimatedSamplingRate))
            else:
                windowSamples = 1
                estimatedSamplingRate = None

            import pandas as pd
            whiskerSmoothed = pd.Series(intervalWhiskerData).rolling(
                window=windowSamples, center=True, min_periods=1
            ).mean().values

            motionData = {
                'times': intervalTimestamps,
                'whiskerMotionEnergy_raw': intervalWhiskerData,
                'whiskerMotionEnergy_smoothed': whiskerSmoothed,
                'smoothing_window': smoothingWindow,
                'estimated_sampling_rate': estimatedSamplingRate
            }

            if self.verbose:
                print(f"✓ Motion energy extracted: {len(intervalTimestamps)} timepoints")
                print(f"  Time range: {intervalTimestamps[0]:.3f} - {intervalTimestamps[-1]:.3f}s")
                print(f"  Data range: {np.nanmin(intervalWhiskerData):.3f} - {np.nanmax(intervalWhiskerData):.3f}")

            return motionData

        except Exception as e:
            if self.verbose:
                print(f"✗ Error extracting motion energy data: {str(e)}")
                import traceback
                traceback.print_exc()
            return None

    def prepareDataForAnimation(self, intervalData: Dict) -> Optional[Dict]:
        """
        Extract all behavioral data for interval and prepare for animation with consistent time base.
        Combines wheel, motion energy extraction and animation data preparation in one method.

        Args:
            intervalData (Dict): Raw interval data from sampleInterval()

        Returns:
            Dict: Animation-ready data with synchronized time bases, or None if preparation fails
        """
        try:
            if self.verbose:
                print("Preparing data for animation...")

            startTime = intervalData['intervalStart']
            endTime = intervalData['intervalEnd']

            # Extract wheel data for this interval using dedicated method
            wheelData = self.extractWheelDataForInterval(startTime, endTime)

            # Extract motion energy data for this interval using dedicated method
            motionData = self.extractMotionEnergyForInterval(startTime, endTime)

            # Prepare animation data structure
            animationData = {
                'metadata': intervalData['sessionInfo'],
                'trial_info': {
                    'trial_index': intervalData['trialIndex'],
                    'start_time': startTime,
                    'end_time': endTime,
                    'duration': intervalData['duration'],
                    'events': intervalData['trialEvents']
                },
                'synchronized_data': {}
            }

            # Define common time base for animation (use video frame times as reference)
            if self.videoTimes is not None:
                videoTimeFilter = (self.videoTimes >= startTime) & (self.videoTimes <= endTime)
                animationTimeBase = self.videoTimes[videoTimeFilter]

                if len(animationTimeBase) > 0:
                    animationData['synchronized_data']['video_times'] = animationTimeBase
                    animationData['synchronized_data']['video_frame_indices'] = np.where(videoTimeFilter)[0]

                    if self.verbose:
                        print(f"  Video time base: {len(animationTimeBase)} frames")
                        print(f"  Frame rate: {len(animationTimeBase) / intervalData['duration']:.1f} fps")
                else:
                    print("! No video frames found in interval")
                    return None
            else:
                print("✗ No video timestamps available for synchronization")
                return None

            # Add pose data with original timestamps (for linear interpolation during animation)
            if intervalData['poseData'] is not None:
                animationData['synchronized_data']['pose'] = {
                    'times': intervalData['poseData']['times'].values,
                    'features': {}
                }

                # Store each pose feature separately for easy access during animation
                for feature in self.requiredPoseFeatures:
                    if feature in intervalData['poseData'].columns and feature != 'times':
                        featureData = intervalData['poseData'][feature].values
                        validMask = ~np.isnan(featureData)

                        animationData['synchronized_data']['pose']['features'][feature] = {
                            'values': featureData,
                            'valid_mask': validMask,
                            'valid_times': intervalData['poseData']['times'].values[validMask],
                            'valid_values': featureData[validMask]
                        }

            # Add wheel data (position-focused)
            if wheelData is not None:
                animationData['synchronized_data']['wheel'] = {
                    'times': wheelData['times'],
                    'position_raw': wheelData['position_raw'],
                }

            # Add motion energy data
            if motionData is not None:
                animationData['synchronized_data']['motion_energy'] = {
                    'times': motionData['times'],
                    'whisker_raw': motionData['whiskerMotionEnergy_raw'],
                    'whisker_smoothed': motionData['whiskerMotionEnergy_smoothed'],
                    'smoothing_window': motionData['smoothing_window'],
                    'sampling_rate': motionData.get('estimated_sampling_rate', None)
                }

            if self.verbose:
                print("✓ Animation data preparation completed")
                print(f"  Data streams prepared: {list(animationData['synchronized_data'].keys())}")

            return animationData

        except Exception as e:
            print(f"✗ Error preparing animation data: {str(e)}")
            return None

    def loadCompleteSession(self) -> bool:
        """
        Complete workflow to load and validate all session data.

        Returns:
            bool: True if all loading steps successful, False otherwise
        """
        if self.verbose:
            print("="*60)
            print("STARTING COMPLETE SESSION LOADING")
            print("="*60)

        steps = [
            ("Loading session data", self.loadSessionData),
            ("Extracting session info", lambda: self.extractSessionInfo() != {}),
            ("Validating trial data", self.validateAndExtractTrialData),
            ("Validating pose data", self.validateAndExtractPoseData),
            ("Extracting behavioral data", self.extractOtherBehavioralData),
            ("Loading video timestamps", self.loadVideoTimestamps)
        ]

        allSuccessful = True
        for stepName, stepFunction in steps:
            if self.verbose:
                print(f"\n{stepName}...")

            success = stepFunction()
            if not success:
                print(f"✗ Failed: {stepName}")
                allSuccessful = False
            elif self.verbose:
                print(f"✓ Completed: {stepName}")

        if self.verbose:
            print("\n" + "="*60)
            if allSuccessful:
                print("SESSION LOADING COMPLETED SUCCESSFULLY")
            else:
                print("SESSION LOADING COMPLETED WITH ERRORS")
            print("="*60)

        return allSuccessful

# Animator Class

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.animation import FuncAnimation
from matplotlib.lines import Line2D
from IPython.display import display, HTML
from datetime import datetime
from typing import Dict, List, Optional
import gc

class IBLAnimationRenderer:
    """
    High-performance animation renderer for IBL behavioral data with video synchronization.
    Displays full trial duration with moving time indicators for optimal performance and context.
    """

    def __init__(self, mode='test', baseDir=None, verbose=True, fps=24, one=None, eid=None, dataLoader=None, cameras=['left', 'right']):
        """Initialize animation renderer with optional DataLoader integration."""
        assert mode in ['test', 'save'], f"Mode must be 'test' or 'save', got {mode}"

        # Validate camera selection
        valid_cameras = ['left', 'right', 'body']
        if not isinstance(cameras, list) or len(cameras) == 0 or len(cameras) > 2:
            raise ValueError("cameras must be a list of 1-2 camera names")
        for cam in cameras:
            if cam not in valid_cameras:
                raise ValueError(f"Invalid camera '{cam}'. Must be one of {valid_cameras}")

        self.mode = mode
        self.baseDir = baseDir or '.'
        self.verbose = verbose
        self.fps = fps
        self.one = one
        self.eid = eid
        self.dataLoader = dataLoader
        self.cameras = cameras  # Store selected cameras

        # Animation elements
        self.lineObjects = {}
        self.timeIndicators = []
        self.videoImages = {}  # Dynamic camera video images
        self.timeTexts = {}    # Dynamic camera time texts
        self.axVideos = {}     # Dynamic camera axes

        # Video loading (dynamic camera setup)
        self.videoUrls = {cam: None for cam in self.cameras}
        self.videoFrameCaches = {cam: {} for cam in self.cameras}
        self.cacheStartFrames = {cam: None for cam in self.cameras}
        self.chunkSize = 100

        # Debug logging
        self.debugLog = []

        # Initialize DataLoader integration if provided
        if self.dataLoader is not None:
            self._validateDataLoader()
            if self.verbose:
                subject = self.dataLoader.sessionInfo.get('subject', 'Unknown')
                print(f"IBL Animation Renderer initialized with DataLoader for subject {subject}")
        elif self.verbose:
            print(f"IBL Animation Renderer initialized (Mode: {mode}, FPS: {fps})")

    def render(self, animationData: Dict) -> None:
        """Create animation from prepared IBL data."""
        try:
            self._extractMetadata(animationData)
            self._initializeVideoLoader()
            self._createFigure()
            self._initializeVideo()
            self._initializePlots(animationData)
            self._initializeEvents(animationData)
            self._initializeTimeIndicators()
            self._createAnimation()
        except Exception as e:
            if self.verbose:
                print(f"Animation error: {str(e)}")
            raise
        finally:
            self._cleanup()

    def _extractMetadata(self, animationData: Dict) -> None:
        """Extract essential timing and session information."""
        if 'synchronized_data' not in animationData or 'video_times' not in animationData['synchronized_data']:
            raise ValueError("Missing video timing data")

        self.trialInfo = animationData['trial_info']
        self.sessionInfo = animationData['metadata']
        self.videoTimes = animationData['synchronized_data']['video_times']
        self.totalFrames = len(self.videoTimes)
        self.syncData = animationData['synchronized_data']

        # CRITICAL DEBUG: Video/Graph Time Alignment
        if self.verbose:
            subject = self.sessionInfo.get('subject', 'Unknown')
            duration = self.trialInfo['duration']
            print(f"Rendering {duration:.1f}s trial for subject {subject} ({self.totalFrames} frames)")

            print(f"\nDEBUG: TIME SYNCHRONIZATION ANALYSIS")
            print(f"  Trial interval: {self.trialInfo['start_time']:.3f} - {self.trialInfo['end_time']:.3f}s")
            print(f"  Video times range: {self.videoTimes[0]:.3f} - {self.videoTimes[-1]:.3f}s")
            print(f"  Video duration: {self.videoTimes[-1] - self.videoTimes[0]:.3f}s")
            print(f"  Total video frames: {self.totalFrames}")
            print(f"  Video frame rate: {self.totalFrames / (self.videoTimes[-1] - self.videoTimes[0]):.2f} fps")

            # Check if video times match trial interval
            if abs(self.videoTimes[0] - self.trialInfo['start_time']) > 0.1:
                print(f"  WARNING: Video start time mismatch!")
                print(f"     Video starts at: {self.videoTimes[0]:.3f}s")
                print(f"     Trial starts at: {self.trialInfo['start_time']:.3f}s")
                print(f"     Difference: {self.videoTimes[0] - self.trialInfo['start_time']:.3f}s")

            if abs(self.videoTimes[-1] - self.trialInfo['end_time']) > 0.1:
                print(f"  WARNING: Video end time mismatch!")
                print(f"     Video ends at: {self.videoTimes[-1]:.3f}s")
                print(f"     Trial ends at: {self.trialInfo['end_time']:.3f}s")
                print(f"     Difference: {self.videoTimes[-1] - self.trialInfo['end_time']:.3f}s")

            # Check video frame indices if available
            if 'video_frame_indices' in self.syncData:
                frame_indices = self.syncData['video_frame_indices']
                print(f"  Video frame indices: {frame_indices[0]} to {frame_indices[-1]} (range: {frame_indices[-1] - frame_indices[0] + 1})")

                # Check for frame index gaps
                if len(frame_indices) > 1:
                    frame_gaps = np.diff(frame_indices)
                    unique_gaps = np.unique(frame_gaps)
                    if len(unique_gaps) > 1:
                        print(f"  WARNING: Non-uniform frame indices!")
                        print(f"     Frame index gaps: {unique_gaps}")
                    else:
                        print(f"  Uniform frame indices (step: {unique_gaps[0]})")

            # Check if DataLoader has original timestamps
            if hasattr(self.dataLoader, 'standardizedTimestamps'):
                print(f"  DataLoader standardized timestamps available for cameras: {list(self.dataLoader.standardizedTimestamps.keys())}")
                for camera in self.cameras:
                    if camera in self.dataLoader.standardizedTimestamps:
                        cam_times = self.dataLoader.standardizedTimestamps[camera]
                        print(f"     {camera}: {len(cam_times)} frames, {cam_times[0]:.3f} - {cam_times[-1]:.3f}s")

            print(f"  Animation will use video times as master clock")

    def _createFigure(self) -> None:
        """Create figure layout with dynamic camera selection and behavioral plots."""
        self.fig = plt.figure(figsize=(20, 11), facecolor='white')

        num_cameras = len(self.cameras)

        if num_cameras == 1:
            # Single camera layout: camera takes full left half
            gs = GridSpec(4, 2,
                         width_ratios=[0.8, 1.2],           # Videos | plots (plots get 20% more space)
                         height_ratios=[1.0, 1.0, 1.0, 1.0], # Equal height for plots
                         hspace=0.2, wspace=0.12,
                         left=0.05, right=0.95, top=0.90, bottom=0.08)

            # Single video display (full left half)
            self.axVideos = {
                self.cameras[0]: self.fig.add_subplot(gs[:, 0])  # Spans all rows
            }

        elif num_cameras == 2:
            # Dual camera layout: equal sized cameras in left half
            gs = GridSpec(4, 2,
                         width_ratios=[0.8, 1.2],           # Videos | plots (plots get 20% more space)
                         height_ratios=[1.0, 1.0, 1.0, 1.0], # Equal height rows
                         hspace=0.2, wspace=0.12,
                         left=0.05, right=0.95, top=0.90, bottom=0.08)

            # Two equal-sized video displays
            self.axVideos = {
                self.cameras[0]: self.fig.add_subplot(gs[0:2, 0]),  # Top half - spans 2 rows
                self.cameras[1]: self.fig.add_subplot(gs[2:4, 0])   # Bottom half - spans 2 rows
            }

        # Set video titles and properties
        for i, camera in enumerate(self.cameras):
            ax = self.axVideos[camera]
            ax.set_title(f'{camera.title()} Camera', fontsize=11, fontweight='bold', pad=10)
            ax.axis('off')

        # Behavioral plots (standardized sizes in right half)
        self.axPlots = {
            'whisker_nose': self.fig.add_subplot(gs[0, 1]),
            'wheel': self.fig.add_subplot(gs[1, 1]),
            'pupil': self.fig.add_subplot(gs[2, 1]),
            'tongue': self.fig.add_subplot(gs[3, 1])
        }

        # Configure plot appearance with axis labels
        for ax in self.axPlots.values():
            ax.grid(True, alpha=0.3, linewidth=0.5)
            ax.set_xlim(self.trialInfo['start_time'], self.trialInfo['end_time'])

        # Enhanced figure title with all required information
        trialIdx = self.trialInfo.get('trial_index', 'Unknown')
        duration = self.trialInfo['duration']
        subject = self.sessionInfo.get('subject', 'Unknown')
        lab = self.sessionInfo.get('lab', 'Unknown')
        self.fig.suptitle(f'IBL Behavioral Analysis - Trial {trialIdx} - Subject {subject} - Lab {lab} - {duration:.1f}s',
                         fontsize=16, fontweight='bold', y=0.95)

    def _initializeVideo(self) -> None:
        """Initialize video displays for selected cameras."""
        placeholderFrame = np.zeros((480, 640), dtype=np.uint8)

        # Initialize video displays dynamically based on selected cameras
        self.videoImages = {}
        self.timeTexts = {}

        for camera in self.cameras:
            ax = self.axVideos[camera]

            # Create video display
            self.videoImages[camera] = ax.imshow(placeholderFrame, cmap='gray', animated=True)

            # Create time text overlay
            self.timeTexts[camera] = ax.text(0.02, 0.98, '', transform=ax.transAxes,
                                           fontsize=11, fontweight='bold', color='white',
                                           bbox=dict(boxstyle='round', facecolor='black', alpha=0.7),
                                           verticalalignment='top', animated=True)

    def _initializePlots(self, animationData: Dict) -> None:
        """Initialize all behavioral plots with full trial duration data."""
        self._initializeWhiskerNosePlot()
        self._initializeWheelPlot()
        self._initializePupilPlot()
        self._initializeTonguePlot()

    def _initializeWhiskerNosePlot(self) -> None:
        """Initialize whisker motion and nose tracking plot."""
        ax = self.axPlots['whisker_nose']
        ax.set_title('Whisker Motion Energy & Nose Position', fontsize=11, fontweight='bold')

        axTwin = ax.twinx()

        # Whisker motion (left axis)
        ax.set_ylabel('Motion Energy (a.u.)', color='red', fontweight='bold')
        ax.tick_params(axis='y', labelcolor='red')

        # Nose position (right axis)
        axTwin.set_ylabel('Nose Position (pixels)', color='blue', fontweight='bold')
        axTwin.tick_params(axis='y', labelcolor='blue')

        lines = {}

        # Plot whisker motion energy data
        if 'motion_energy' in self.syncData:
            motionData = self.syncData['motion_energy']
            times = motionData.get('times', [])

            if 'whisker_raw' in motionData and len(times) > 0:
                rawValues = motionData['whisker_raw']
                lines['whisker_raw'] = ax.plot(times, rawValues, color='yellow',
                                              linewidth=1.5, label='Whisker Raw', alpha=0.8)[0]

            if 'whisker_smoothed' in motionData and len(times) > 0:
                smoothedValues = motionData['whisker_smoothed']
                lines['whisker_smooth'] = ax.plot(times, smoothedValues, color='red',
                                                 linewidth=2.5, label='Whisker Smooth')[0]

            # Set y-limits based on data
            if lines:
                allValues = []
                for key in ['whisker_raw', 'whisker_smoothed']:
                    if key in motionData:
                        allValues.extend(motionData[key])

                if allValues:
                    vmin, vmax = np.nanmin(allValues), np.nanmax(allValues)
                    vrange = vmax - vmin
                    ax.set_ylim(vmin - 0.1 * vrange, vmax + 0.1 * vrange)

        # Plot nose tracking data
        if 'pose' in self.syncData and 'features' in self.syncData['pose']:
            poseData = self.syncData['pose']
            poseTimes = poseData.get('times', [])
            features = poseData.get('features', {})

            if 'nose_tip_x' in features and len(poseTimes) > 0:
                noseX = features['nose_tip_x']['values']
                validMask = features['nose_tip_x']['valid_mask']
                validTimes = poseTimes[validMask]
                validValues = noseX[validMask]
                if len(validValues) > 0:
                    lines['nose_x'] = axTwin.plot(validTimes, validValues, color='blue',
                                                 linewidth=2, label='Nose X')[0]

            if 'nose_tip_y' in features and len(poseTimes) > 0:
                noseY = features['nose_tip_y']['values']
                validMask = features['nose_tip_y']['valid_mask']
                validTimes = poseTimes[validMask]
                validValues = noseY[validMask]
                if len(validValues) > 0:
                    lines['nose_y'] = axTwin.plot(validTimes, validValues, color='cyan',
                                                 linewidth=2, label='Nose Y')[0]

            # Set y-limits for nose position with min-max standardization
            noseValues = []
            for feature in ['nose_tip_x', 'nose_tip_y']:
                if feature in features:
                    values = features[feature]['values']
                    validMask = features[feature]['valid_mask']
                    noseValues.extend(values[validMask])

            if noseValues:
                noseMin, noseMax = np.nanmin(noseValues), np.nanmax(noseValues)
                noseRange = noseMax - noseMin
                if noseRange > 0:
                    axTwin.set_ylim(noseMin - 0.1 * noseRange, noseMax + 0.1 * noseRange)
                else:
                    axTwin.set_ylim(noseMin - 10, noseMax + 10)

        # Add legends
        if any('whisker' in key for key in lines.keys()):
            ax.legend(loc='upper left', fontsize=10)
        if any('nose' in key for key in lines.keys()):
            axTwin.legend(loc='upper right', fontsize=10)

        self.lineObjects['whisker_nose'] = lines

    def _initializeWheelPlot(self) -> None:
        """Initialize wheel position plot."""
        ax = self.axPlots['wheel']
        ax.set_title('Wheel Position', fontsize=11, fontweight='bold')
        ax.set_ylabel('Position (radians)', color='darkblue', fontweight='bold')
        ax.tick_params(axis='y', labelcolor='darkblue')

        lines = {}

        if 'wheel' in self.syncData:
            wheelData = self.syncData['wheel']
            times = wheelData.get('times', [])

            # Display wheel position instead of velocity/acceleration
            if 'position_raw' in wheelData and len(times) > 0:
                position = wheelData['position_raw']

                # Handle NaN values in position data
                if np.any(~np.isnan(position)):
                    lines['position'] = ax.plot(times, position, color='darkblue',
                                               linewidth=2.5, label='Wheel Position')[0]

                    # Set y-limits based on actual position data
                    validPosition = position[~np.isnan(position)]
                    if len(validPosition) > 0:
                        pmin, pmax = np.min(validPosition), np.max(validPosition)
                        prange = pmax - pmin
                        if prange > 0:
                            ax.set_ylim(pmin - 0.1 * prange, pmax + 0.1 * prange)
                        else:
                            ax.set_ylim(pmin - 1, pmax + 1)
                    else:
                        ax.set_ylim(-10, 10)  # Default range
                else:
                    # No valid position data
                    ax.text(0.5, 0.5, 'No valid wheel position data',
                           transform=ax.transAxes, ha='center', va='center',
                           fontsize=10, color='gray')
                    ax.set_ylim(-1, 1)

        # Add legend
        if lines:
            ax.legend(loc='upper right', fontsize=10)

        self.lineObjects['wheel'] = lines

    def _initializePupilPlot(self) -> None:
        """Initialize pupil tracking plot."""
        ax = self.axPlots['pupil']
        ax.set_title('Pupil Tracking', fontsize=11, fontweight='bold')

        axTwin = ax.twinx()

        lines = {}

        if 'pose' in self.syncData and 'features' in self.syncData['pose']:
            poseData = self.syncData['pose']
            poseTimes = poseData.get('times', [])
            features = poseData.get('features', {})

            # Pupil X (left axis)
            if 'pupil_top_r_x' in features and len(poseTimes) > 0:
                pupilX = features['pupil_top_r_x']['values']
                validMask = features['pupil_top_r_x']['valid_mask']
                validTimes = poseTimes[validMask]
                validValues = pupilX[validMask]
                if len(validValues) > 0:
                    lines['pupil_x'] = ax.plot(validTimes, validValues, color='darkblue',
                                              linewidth=2, label='Pupil X')[0]

                    # Set y-limits for X axis
                    pupilXMin, pupilXMax = np.nanmin(validValues), np.nanmax(validValues)
                    pupilXRange = pupilXMax - pupilXMin
                    if pupilXRange > 0:
                        ax.set_ylim(pupilXMin - 0.1 * pupilXRange, pupilXMax + 0.1 * pupilXRange)
                    else:
                        ax.set_ylim(pupilXMin - 10, pupilXMax + 10)

                    ax.set_ylabel('Pupil X Position (pixels)', color='darkblue', fontweight='bold')
                    ax.tick_params(axis='y', labelcolor='darkblue')

            # Pupil Y (right axis)
            if 'pupil_top_r_y' in features and len(poseTimes) > 0:
                pupilY = features['pupil_top_r_y']['values']
                validMask = features['pupil_top_r_y']['valid_mask']
                validTimes = poseTimes[validMask]
                validValues = pupilY[validMask]
                if len(validValues) > 0:
                    lines['pupil_y'] = axTwin.plot(validTimes, validValues, color='violet',
                                                  linewidth=2, label='Pupil Y')[0]

                    # Set y-limits for Y axis
                    pupilYMin, pupilYMax = np.nanmin(validValues), np.nanmax(validValues)
                    pupilYRange = pupilYMax - pupilYMin
                    if pupilYRange > 0:
                        axTwin.set_ylim(pupilYMin - 0.1 * pupilYRange, pupilYMax + 0.1 * pupilYRange)
                    else:
                        axTwin.set_ylim(pupilYMin - 10, pupilYMax + 10)

                    axTwin.set_ylabel('Pupil Y Position (pixels)', color='violet', fontweight='bold')
                    axTwin.tick_params(axis='y', labelcolor='violet')

        # Add legends
        if any('pupil_x' in key for key in lines.keys()):
            ax.legend(loc='upper left', fontsize=10)
        if any('pupil_y' in key for key in lines.keys()):
            axTwin.legend(loc='upper right', fontsize=10)

        self.lineObjects['pupil'] = lines

    def _initializeTonguePlot(self) -> None:
        """Initialize tongue tracking plot."""
        ax = self.axPlots['tongue']
        ax.set_title('Tongue Tracking', fontsize=11, fontweight='bold')
        ax.set_ylabel('Tongue Position (pixels)', fontweight='bold')

        lines = {}

        if 'pose' in self.syncData and 'features' in self.syncData['pose']:
            poseData = self.syncData['pose']
            poseTimes = poseData.get('times', [])
            features = poseData.get('features', {})

            tongueValues = []

            if 'tongue_end_l_x' in features and len(poseTimes) > 0:
                tongueX = features['tongue_end_l_x']['values']
                validMask = features['tongue_end_l_x']['valid_mask']
                validTimes = poseTimes[validMask]
                validValues = tongueX[validMask]
                if len(validValues) > 0:
                    lines['tongue_x'] = ax.plot(validTimes, validValues, color='magenta',
                                               linewidth=2, label='Tongue X')[0]
                    tongueValues.extend(validValues)

            if 'tongue_end_l_y' in features and len(poseTimes) > 0:
                tongueY = features['tongue_end_l_y']['values']
                validMask = features['tongue_end_l_y']['valid_mask']
                validTimes = poseTimes[validMask]
                validValues = tongueY[validMask]
                if len(validValues) > 0:
                    lines['tongue_y'] = ax.plot(validTimes, validValues, color='brown',
                                               linewidth=2, label='Tongue Y')[0]
                    tongueValues.extend(validValues)

            # Set y-limits based on actual data range
            if tongueValues:
                tongueMin, tongueMax = np.nanmin(tongueValues), np.nanmax(tongueValues)
                tongueRange = tongueMax - tongueMin
                if tongueRange > 0:
                    ax.set_ylim(tongueMin - 0.1 * tongueRange, tongueMax + 0.1 * tongueRange)
                else:
                    ax.set_ylim(tongueMin - 10, tongueMax + 10)
            else:
                ax.set_ylim(0, 480)

        ax.legend(loc='upper right', fontsize=10)
        self.lineObjects['tongue'] = lines

    def _initializeEvents(self, animationData: Dict) -> None:
        """Initialize experimental event markers."""
        # Only show requested events: stimOn, stimOff, firstMovement, Response, Feedback
        eventStyles = {
            'stimOn': {'color': 'red', 'linestyle': '-', 'linewidth': 2, 'label': 'stimOn'},
            'stimOff': {'color': 'orange', 'linestyle': '-', 'linewidth': 2, 'label': 'stimOff'},
            'firstMovement': {'color': 'cyan', 'linestyle': '-.', 'linewidth': 2, 'label': 'firstMovement'},
            'response': {'color': 'blue', 'linestyle': '-', 'linewidth': 2, 'label': 'Response'},
            'feedback': {'color': 'magenta', 'linestyle': ':', 'linewidth': 2, 'label': 'Feedback'}
        }

        trialEvents = animationData['trial_info'].get('events', {})
        startTime = self.trialInfo['start_time']
        endTime = self.trialInfo['end_time']

        debugInfo = {
            'trial_interval': {'start': startTime, 'end': endTime},
            'available_event_keys': list(trialEvents.keys()),
            'events_processed': {}
        }

        self._log_debug(f"DEBUGGING EVENT EXTRACTION:")
        self._log_debug(f"Trial interval: {startTime:.3f} - {endTime:.3f}s")
        self._log_debug(f"Available trial event keys: {list(trialEvents.keys())}")

        legendHandles = []

        for eventName, eventStyle in eventStyles.items():
            eventTimes = self._extractEventTimes(trialEvents, eventName)

            self._log_debug(f"{eventName}:")
            self._log_debug(f"  Raw event times: {eventTimes}")

            eventInfo = {
                'raw_times': eventTimes,
                'valid_times': [],
                'markers_added': 0
            }

            if eventTimes:
                validEventTimes = [t for t in eventTimes if startTime <= t <= endTime]
                eventInfo['valid_times'] = validEventTimes

                self._log_debug(f"  Valid event times (within interval): {validEventTimes}")

                if validEventTimes:
                    for ax in self.axPlots.values():
                        for eventTime in validEventTimes:
                            ax.axvline(x=eventTime, alpha=0.8,
                                     **{k: v for k, v in eventStyle.items() if k != 'label'})

                    legendHandles.append(Line2D([0], [0], **eventStyle))
                    eventInfo['markers_added'] = len(validEventTimes)

                    self._log_debug(f"  Added {len(validEventTimes)} event markers")
                else:
                    self._log_debug(f"  No events within trial interval")
            else:
                self._log_debug(f"  No event times found")

            debugInfo['events_processed'][eventName] = eventInfo

        # Add legend at bottom center to not obstruct views
        if legendHandles:
            self.fig.legend(handles=legendHandles,
                           loc='lower center',
                           bbox_to_anchor=(0.5, 0.01),
                           ncol=5, fontsize=10,
                           frameon=True,
                           title='Events',
                           title_fontsize=11)

            self._log_debug(f"Added legend for {len(legendHandles)} event types")
        else:
            self._log_debug(f"No valid events found - no legend added")

        # Store debug info for save mode
        if self.mode == 'save':
            self.eventDebugInfo = debugInfo

    def _extractEventTimes(self, trialEvents: Dict, eventName: str) -> List[float]:
        """Extract timing information for specific event type."""
        self._log_debug(f"  Extracting {eventName}:")

        # Check direct event timing storage
        directKey = f"{eventName}_times"
        self._log_debug(f"    Checking direct key: {directKey}")
        if directKey in trialEvents:
            eventData = trialEvents[directKey]
            self._log_debug(f"    Found data: {eventData} (type: {type(eventData)})")
            if isinstance(eventData, (list, np.ndarray)):
                validTimes = [float(t) for t in eventData if not np.isnan(float(t))]
                self._log_debug(f"    Valid times from list: {validTimes}")
                return validTimes
            elif not np.isnan(float(eventData)):
                validTime = float(eventData)
                self._log_debug(f"    Valid time from scalar: {validTime}")
                return [validTime]

        # Check alternative naming conventions
        altKeys = [eventName, f"{eventName}Times", f"{eventName.lower()}_times"]
        self._log_debug(f"    Checking alternative keys: {altKeys}")

        for key in altKeys:
            if key in trialEvents:
                eventData = trialEvents[key]
                self._log_debug(f"    Found {key}: {eventData} (type: {type(eventData)})")
                if isinstance(eventData, (list, np.ndarray)):
                    validTimes = [float(t) for t in eventData if not np.isnan(float(t))]
                    self._log_debug(f"    Valid times from {key}: {validTimes}")
                    return validTimes
                elif not np.isnan(float(eventData)):
                    validTime = float(eventData)
                    self._log_debug(f"    Valid time from {key}: {validTime}")
                    return [validTime]

        self._log_debug(f"    No valid times found for {eventName}")
        return []

    def _log_debug(self, message):
        """Add debug message to log."""
        if self.mode == 'save':
            self.debugLog.append(message)
        if self.verbose:
            print(message)

    def _initializeVideoLoader(self) -> None:
        """Initialize video loading system for selected cameras."""
        if self.one is None or self.eid is None:
            self._log_debug("No ONE instance or EID provided - video will use placeholders")
            return

        try:
            from ibllib.io import video
            videoUrls = video.url_from_eid(self.eid, one=self.one)

            # Initialize only selected cameras
            for camera in self.cameras:
                self.videoUrls[camera] = videoUrls.get(camera)

            cameras_found = [cam for cam in self.cameras if self.videoUrls[cam] is not None]
            if cameras_found:
                self._log_debug(f"Video URLs loaded for selected cameras: {cameras_found}")
            else:
                self._log_debug("No video URLs found for selected cameras")

        except Exception as e:
            self._log_debug(f"Video initialization failed: {e}")
            # Reset to None for all selected cameras
            for camera in self.cameras:
                self.videoUrls[camera] = None

    def _getCameraFrameForTime(self, camera: str, target_time: float):
        """
        Get the best camera frame index for a specific time using interpolation.
        This respects the camera's original frame rate instead of forcing alignment.
        """
        if not hasattr(self.dataLoader, 'one') or not hasattr(self.dataLoader, 'eid'):
            return None, None, float('inf')

        try:
            # Get original camera timestamps directly from ONE
            from one.api import ONE
            one = self.dataLoader.one
            eid = self.dataLoader.eid

            # Load raw camera timestamps
            camera_times = one.load_dataset(eid, f'_ibl_{camera}Camera.times.npy')

            # Find closest frame to target time
            time_diffs = np.abs(camera_times - target_time)
            closest_idx = np.argmin(time_diffs)
            actual_time = camera_times[closest_idx]
            time_error = abs(actual_time - target_time)

            if self.verbose and hasattr(self, '_first_frame_debug') and camera not in self._first_frame_debug:
                print(f"Camera {camera} raw timing: {len(camera_times)} frames, {camera_times[0]:.3f}-{camera_times[-1]:.3f}s")
                if not hasattr(self, '_first_frame_debug'):
                    self._first_frame_debug = set()
                self._first_frame_debug.add(camera)

            return closest_idx, actual_time, time_error

        except Exception as e:
            if self.verbose:
                print(f"Failed to get raw timestamps for {camera}: {e}")
            return None, None, float('inf')

    def _loadVideoChunk(self, startFrame: int, camera: str = 'left') -> None:
        """Load chunk of video frames using correct timing approach."""
        if camera not in self.videoUrls or self.videoUrls[camera] is None:
            return

        try:
            from ibllib.io import video

            # NEW APPROACH: Use animation timeline to get correct camera frames
            animation_start_idx = startFrame
            animation_end_idx = min(startFrame + self.chunkSize, self.totalFrames)

            # Get the times for these animation frames
            animation_times = self.videoTimes[animation_start_idx:animation_end_idx]

            # Map each animation time to the best camera frame
            camera_frame_indices = []
            timing_errors = []

            for anim_time in animation_times:
                cam_frame_idx, actual_time, time_error = self._getCameraFrameForTime(camera, anim_time)
                if cam_frame_idx is not None:
                    camera_frame_indices.append(cam_frame_idx)
                    timing_errors.append(time_error)
                else:
                    # Fallback: estimate frame index
                    if hasattr(self, '_fallback_camera_start'):
                        cam_start_time = self._fallback_camera_start.get(camera, anim_time)
                    else:
                        cam_start_time = anim_time
                    estimated_idx = int((anim_time - cam_start_time) * 60)  # Assume ~60fps
                    camera_frame_indices.append(max(0, estimated_idx))
                    timing_errors.append(0.1)  # Large error for fallback

            if self.verbose and startFrame == 0:
                print(f"\nDEBUG: Corrected video loading for {camera}")
                print(f"  Animation frames: {animation_start_idx} to {animation_end_idx-1}")
                print(f"  Animation times: {animation_times[0]:.3f}s to {animation_times[-1]:.3f}s")
                if camera_frame_indices:
                    print(f"  {camera} frame indices: {camera_frame_indices[0]} to {camera_frame_indices[-1]}")
                    avg_error = np.mean(timing_errors) * 1000  # Convert to ms
                    max_error = np.max(timing_errors) * 1000
                    print(f"  Timing errors: avg={avg_error:.1f}ms, max={max_error:.1f}ms")

            # Load the mapped camera frames
            if camera_frame_indices:
                frames = video.get_video_frames_preload(self.videoUrls[camera], camera_frame_indices)

                # Store frames in cache with animation frame indices as keys
                for i, frame in enumerate(frames):
                    self.videoFrameCaches[camera][startFrame + i] = frame

                self.cacheStartFrames[camera] = startFrame

                if self.verbose and startFrame == 0:
                    print(f"Loaded {camera}: {len(frames)} frames with corrected timing")
            else:
                if self.verbose:
                    print(f"No valid frame indices for {camera}")

        except Exception as e:
            if self.verbose:
                print(f"Failed to load {camera} camera chunk at frame {startFrame}: {e}")

    def _getVideoFrame(self, frameIdx: int, camera: str = 'left') -> np.ndarray:
        """Get video frame using corrected timing approach."""

        # DEBUG: Show corrected timing for first few frames
        if self.verbose and frameIdx < 3:
            current_time = self.videoTimes[frameIdx]
            cam_frame_idx, actual_time, time_error = self._getCameraFrameForTime(camera, current_time)

            print(f"CORRECTED: Camera {camera} frame {frameIdx}: AnimTime={current_time:.3f}s")
            if cam_frame_idx is not None:
                print(f"   Best camera frame: {cam_frame_idx}, actual time: {actual_time:.3f}s")
                print(f"   Timing error: {time_error*1000:.1f}ms")
                if time_error < 0.05:  # Less than 50ms
                    print(f"   Excellent timing alignment")
                elif time_error < 0.1:  # Less than 100ms
                    print(f"   Good timing alignment")
                else:
                    print(f"   WARNING: Large timing error")
            else:
                print(f"   Failed to find camera frame")

        # Check if we need to load a new chunk for this camera
        if (self.cacheStartFrames[camera] is None or
            frameIdx < self.cacheStartFrames[camera] or
            frameIdx >= self.cacheStartFrames[camera] + self.chunkSize):

            # Calculate chunk start (align to chunk boundaries)
            chunkStart = (frameIdx // self.chunkSize) * self.chunkSize
            self._loadVideoChunk(chunkStart, camera)

        # Return frame from camera-specific cache or placeholder
        if frameIdx in self.videoFrameCaches[camera]:
            frame = self.videoFrameCaches[camera][frameIdx]
            # Handle frame format
            if frame.ndim == 3 and frame.shape[2] == 1:
                frame = frame[..., 0]
            return frame
        else:
            # Fallback placeholder with debug info
            if self.verbose and frameIdx < 5:
                print(f"Using placeholder frame for {camera} at animation frame {frameIdx}")
            return np.random.randint(0, 255, (480, 640), dtype=np.uint8)

    def _initializeTimeIndicators(self) -> None:
        """Initialize vertical time indicator lines."""
        self.timeIndicators = []
        startTime = self.trialInfo['start_time']

        for ax in self.axPlots.values():
            timeLine = ax.axvline(x=startTime, color='red', linewidth=3,
                                 alpha=0.8, animated=True, zorder=10)
            self.timeIndicators.append(timeLine)

    def _createAnimation(self) -> None:
        """Create and display animation."""
        animation = FuncAnimation(
            self.fig,
            self._updateFrame,
            frames=self.totalFrames,
            interval=1000 / self.fps,
            blit=True,
            repeat=False
        )

        self._saveOrDisplay(animation)

    def _updateFrame(self, frameIdx: int) -> List:
        """Update frame - move time indicators and update video displays."""
        currentTime = self.videoTimes[frameIdx]
        updatedElements = []

        # DEBUG: Show timing details for first few frames
        if self.verbose and frameIdx < 5:
            video_frame_idx = self.syncData['video_frame_indices'][frameIdx] if 'video_frame_indices' in self.syncData else frameIdx
            print(f"Frame {frameIdx}: Time={currentTime:.3f}s, VideoFrameIdx={video_frame_idx}")

            # Check if this matches expected trial timing
            trial_progress = (currentTime - self.trialInfo['start_time']) / self.trialInfo['duration']
            print(f"   Trial progress: {trial_progress*100:.1f}% ({currentTime:.3f}s / {self.trialInfo['duration']:.3f}s)")

        # Update video frames for all selected cameras
        for camera in self.cameras:
            frame = self._getVideoFrame(frameIdx, camera)
            self.videoImages[camera].set_array(frame)
            updatedElements.append(self.videoImages[camera])

        # Update time displays on all videos
        timeText = f'Time: {currentTime:.2f}s'
        for camera in self.cameras:
            self.timeTexts[camera].set_text(timeText)
            updatedElements.append(self.timeTexts[camera])

        # Move time indicators across all plots
        for timeLine in self.timeIndicators:
            timeLine.set_xdata([currentTime, currentTime])
        updatedElements.extend(self.timeIndicators)

        return updatedElements

    def _saveOrDisplay(self, animation: FuncAnimation) -> None:
        """Save or display animation."""
        if self.mode == 'test':
            self._log_debug("Displaying animation...")
            display(HTML(animation.to_jshtml()))
        else:
            self._log_debug("Saving animation...")

            sessionId = self.sessionInfo.get('experimentId', 'unknown')
            outDir = os.path.join(self.baseDir, f"session_{sessionId}")

            # Create directory if it doesn't exist
            os.makedirs(outDir, exist_ok=True)

            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            trialIdx = self.trialInfo.get('trial_index', 'unknown')
            filename = f"trial_{trialIdx}_{timestamp}.mp4"
            filepath = os.path.join(outDir, filename)

            animation.save(filepath, writer='ffmpeg', fps=self.fps,
                          bitrate=5000, codec='libx264')

            self._log_debug(f"Animation saved to: {filepath}")

            # Save debug log to JSON file
            if hasattr(self, 'eventDebugInfo'):
                debugFilename = f"trial_{trialIdx}_{timestamp}_debug.json"
                debugFilepath = os.path.join(outDir, debugFilename)

                debugData = {
                    'trial_info': self.trialInfo,
                    'session_info': self.sessionInfo,
                    'event_debug': self.eventDebugInfo,
                    'debug_log': self.debugLog,
                    'video_file': filename
                }

                import json
                with open(debugFilepath, 'w') as f:
                    json.dump(debugData, f, indent=2, default=str)

                self._log_debug(f"Debug info saved to: {debugFilepath}")

    def _cleanup(self) -> None:
        """Clean up resources."""
        gc.collect()

    # DataLoader Integration Methods
    def _validateDataLoader(self) -> None:
        """Validate DataLoader state."""
        if isinstance(self.dataLoader, type):
            raise TypeError("Expected DataLoader instance, received class.")

        if not hasattr(self.dataLoader, 'sessionInfo') or not self.dataLoader.sessionInfo:
            raise AttributeError("DataLoader has not loaded session data.")

        if not hasattr(self.dataLoader, 'trialData') or self.dataLoader.trialData is None:
            raise ValueError("DataLoader does not contain trial data.")

    def animateRandomInterval(self) -> bool:
        """Create animation of random trial interval."""
        if self.dataLoader is None:
            raise ValueError("DataLoader required for this method. Initialize with dataLoader parameter.")

        try:
            if self.verbose:
                print("Creating random interval animation...")

            animationData = self._sampleIntervalAdaptive()
            if animationData is None:
                return False

            self.render(animationData)
            if self.verbose:
                print("Animation completed!")
            return True

        except Exception as e:
            if self.verbose:
                print(f"Animation failed: {str(e)}")
            return False

    def animateSpecificTrial(self, trialIndex: int) -> bool:
        """Create animation of specific trial."""
        if self.dataLoader is None:
            raise ValueError("DataLoader required for this method. Initialize with dataLoader parameter.")

        try:
            if trialIndex >= len(self.dataLoader.trialData):
                if self.verbose:
                    print(f"Trial {trialIndex} not found")
                return False

            if self.verbose:
                print(f"Animating trial {trialIndex}...")

            selectedTrial = self.dataLoader.trialData.iloc[trialIndex]
            intervalStart = selectedTrial['intervals_0']
            intervalEnd = selectedTrial['intervals_1']

            intervalData = {
                'trialIndex': trialIndex,
                'intervalStart': intervalStart,
                'intervalEnd': intervalEnd,
                'duration': intervalEnd - intervalStart,
                'trialEvents': selectedTrial.to_dict(),
                'sessionInfo': self.dataLoader.sessionInfo
            }

            # Extract behavioral data
            if hasattr(self.dataLoader, 'extractPoseDataForInterval'):
                intervalData['poseData'] = self.dataLoader.extractPoseDataForInterval(intervalStart, intervalEnd)
            if hasattr(self.dataLoader, 'extractWheelDataForInterval'):
                intervalData['wheelData'] = self.dataLoader.extractWheelDataForInterval(intervalStart, intervalEnd)
            if hasattr(self.dataLoader, 'extractMotionEnergyForInterval'):
                intervalData['motionEnergyData'] = self.dataLoader.extractMotionEnergyForInterval(intervalStart, intervalEnd)

            animationData = self._prepareAnimationDataLocally(intervalData)
            if animationData is None:
                return False

            self.render(animationData)
            if self.verbose:
                print(f"Trial {trialIndex} completed!")
            return True

        except Exception as e:
            if self.verbose:
                print(f"Failed to animate trial {trialIndex}: {str(e)}")
            return False

    def _sampleIntervalAdaptive(self):
        """Sample interval with adaptive DataLoader compatibility."""
        if hasattr(self.dataLoader, 'prepareDataForAnimation'):
            try:
                import random
                trialIdx = random.randint(0, len(self.dataLoader.trialData) - 1)
                rawData = self.dataLoader.sampleInterval(trialIdx)
                return self.dataLoader.prepareDataForAnimation(rawData) if rawData else None
            except:
                pass

        import random
        trialIdx = random.randint(0, len(self.dataLoader.trialData) - 1)
        rawData = self.dataLoader.sampleInterval(trialIdx)
        return self._prepareAnimationDataLocally(rawData) if rawData else None

    def _prepareAnimationDataLocally(self, rawIntervalData):
        """Prepare animation data locally."""
        try:
            startTime = rawIntervalData['intervalStart']
            endTime = rawIntervalData['intervalEnd']

            animationData = {
                'metadata': rawIntervalData['sessionInfo'],
                'trial_info': {
                    'trial_index': rawIntervalData['trialIndex'],
                    'start_time': startTime,
                    'end_time': endTime,
                    'duration': rawIntervalData['duration'],
                    'events': rawIntervalData['trialEvents']
                },
                'synchronized_data': {}
            }

            # Process video timing with DEBUG
            if hasattr(self.dataLoader, 'videoTimes') and self.dataLoader.videoTimes is not None:
                videoFilter = (self.dataLoader.videoTimes >= startTime) & (self.dataLoader.videoTimes <= endTime)
                videoTimes = self.dataLoader.videoTimes[videoFilter]

                if len(videoTimes) > 0:
                    animationData['synchronized_data']['video_times'] = videoTimes

                    # CRITICAL FIX: Get actual frame indices, not relative indices
                    frame_indices = np.where(videoFilter)[0]
                    animationData['synchronized_data']['video_frame_indices'] = frame_indices

                    if self.verbose:
                        print(f"\nDEBUG: Video timing processing")
                        print(f"  DataLoader video times: {len(self.dataLoader.videoTimes)} total frames")
                        print(f"  DataLoader time range: {self.dataLoader.videoTimes[0]:.3f} - {self.dataLoader.videoTimes[-1]:.3f}s")
                        print(f"  Trial interval: {startTime:.3f} - {endTime:.3f}s")
                        print(f"  Filtered video times: {len(videoTimes)} frames")
                        print(f"  Filtered time range: {videoTimes[0]:.3f} - {videoTimes[-1]:.3f}s")
                        print(f"  Frame indices: {frame_indices[0]} to {frame_indices[-1]} (step: {frame_indices[1]-frame_indices[0] if len(frame_indices)>1 else 'N/A'})")

                        # Check for different camera timestamps
                        if hasattr(self.dataLoader, 'standardizedTimestamps'):
                            print(f"  Camera timestamp analysis:")
                            for cam_name, cam_times in self.dataLoader.standardizedTimestamps.items():
                                cam_filter = (cam_times >= startTime) & (cam_times <= endTime)
                                cam_in_trial = cam_times[cam_filter]
                                cam_indices = np.where(cam_filter)[0]
                                print(f"    {cam_name}: {len(cam_in_trial)} frames, indices {cam_indices[0] if len(cam_indices)>0 else 'N/A'} to {cam_indices[-1] if len(cam_indices)>0 else 'N/A'}")

                                # Check if camera times match our video times
                                if cam_name == 'left' and len(cam_in_trial) > 0:
                                    time_diff = abs(cam_in_trial[0] - videoTimes[0]) + abs(cam_in_trial[-1] - videoTimes[-1])
                                    if time_diff > 0.1:
                                        print(f"    WARNING: {cam_name} camera times don't match video times (diff: {time_diff:.3f}s)")
                                    else:
                                        print(f"    {cam_name} camera times match video times")

            # Add behavioral data
            self._addPoseData(animationData, rawIntervalData)
            self._addWheelData(animationData, rawIntervalData)
            self._addMotionEnergyData(animationData, rawIntervalData)

            return animationData

        except Exception as e:
            if self.verbose:
                print(f"Error preparing animation data: {str(e)}")
            return None

    def _addPoseData(self, animationData, rawIntervalData):
        """Add pose data to animation data structure."""
        if 'poseData' in rawIntervalData and rawIntervalData['poseData'] is not None:
            poseData = rawIntervalData['poseData']
            animationData['synchronized_data']['pose'] = {
                'times': poseData['times'].values,
                'features': {}
            }

            # Store each pose feature
            for feature in poseData.columns:
                if feature != 'times':
                    featureData = poseData[feature].values
                    validMask = ~np.isnan(featureData)

                    animationData['synchronized_data']['pose']['features'][feature] = {
                        'values': featureData,
                        'valid_mask': validMask,
                        'valid_times': poseData['times'].values[validMask],
                        'valid_values': featureData[validMask]
                    }

    def _addWheelData(self, animationData, rawIntervalData):
        """Add wheel data to animation data structure (position-focused)."""
        if 'wheelData' in rawIntervalData and rawIntervalData['wheelData'] is not None:
            wheelData = rawIntervalData['wheelData']
            animationData['synchronized_data']['wheel'] = {
                'times': wheelData['times'],
                'position_raw': wheelData.get('position_raw'),
            }

    def _addMotionEnergyData(self, animationData, rawIntervalData):
        """Add motion energy data to animation data structure."""
        if 'motionEnergyData' in rawIntervalData and rawIntervalData['motionEnergyData'] is not None:
            motionData = rawIntervalData['motionEnergyData']
            animationData['synchronized_data']['motion_energy'] = {
                'times': motionData['times'],
                'whisker_raw': motionData.get('whiskerMotionEnergy_raw'),
                'whisker_smoothed': motionData.get('whiskerMotionEnergy_smoothed')
            }


# Complete workflow wrapper function
def animateIBLSession(eid, mode='test', fps=24, trialIndex=None, verbose=True, baseDir=None):
    """
    Complete workflow to create IBL behavioral animations.

    Args:
        eid (str): IBL experiment ID
        mode (str): 'test' for inline display, 'save' for MP4 output
        fps (int): Animation frame rate
        trialIndex (int, optional): Specific trial to animate. If None, random trial
        verbose (bool): Enable detailed logging
        baseDir (str, optional): Base directory for saving (only used in save mode)

    Returns:
        bool: True if animation succeeded, False otherwise
    """
    try:
        if verbose:
            print("="*60)
            print("IBL ANIMATION WORKFLOW")
            print("="*60)
            print(f"Experiment ID: {eid}")
            print(f"Mode: {mode}, FPS: {fps}")
            if trialIndex is not None:
                print(f"Trial: {trialIndex}")
            else:
                print("Trial: Random")

        # Step 1: Initialize DataLoader
        if verbose:
            print("\n1. Initializing DataLoader...")
        dataLoader = IBLDataLoader(eid=eid, verbose=verbose)

        # Step 2: Load session data
        if verbose:
            print("\n2. Loading session data...")
        loadSuccess = dataLoader.loadCompleteSession()
        if not loadSuccess:
            print("Session loading failed")
            return False

        # Step 3: Create animation renderer
        if verbose:
            print("\n3. Creating animation renderer...")
        renderer = createAnimationRenderer(dataLoader, mode=mode, fps=fps, verbose=verbose, baseDir=baseDir)

        # Step 4: Create animation
        if verbose:
            print("\n4. Creating animation...")
        if trialIndex is not None:
            success = renderer.animateSpecificTrial(trialIndex)
        else:
            success = renderer.animateRandomInterval()

        if verbose:
            if success:
                print("\nAnimation workflow completed successfully!")
            else:
                print("\nAnimation workflow failed")

        return success

    except Exception as e:
        if verbose:
            print(f"\nWorkflow error: {str(e)}")
        return False

# Convenience functions for easy use
def createAnimationRenderer(dataLoader, mode='test', fps=24, verbose=True, baseDir=None, cameras=['left', 'right']):
    """
    Create animation renderer with DataLoader integration.

    Args:
        dataLoader: IBLDataLoader instance with loaded session data
        mode (str): 'test' for inline display, 'save' for MP4 output
        fps (int): Animation frame rate
        verbose (bool): Enable status messages
        baseDir (str, optional): Base directory for saving
        cameras (list): List of 1-2 cameras from ['left', 'right', 'body']

    Returns:
        IBLAnimationRenderer: Ready-to-use animation renderer with DataLoader integration
    """
    return IBLAnimationRenderer(mode=mode, fps=fps, verbose=verbose,
                               one=dataLoader.one, eid=dataLoader.eid,
                               baseDir=baseDir, dataLoader=dataLoader, cameras=cameras)

def quickAnimate(dataLoader, mode='test', fps=24, cameras=['left', 'right']):
    """
    Quick animation of random interval - one-line convenience function.

    Args:
        dataLoader: IBLDataLoader instance with loaded session data
        mode (str): 'test' for inline display, 'save' for MP4 output
        fps (int): Animation frame rate
        cameras (list): List of 1-2 cameras from ['left', 'right', 'body']

    Returns:
        bool: True if animation succeeded, False otherwise
    """
    renderer = createAnimationRenderer(dataLoader, mode=mode, fps=fps, cameras=cameras)
    return renderer.animateRandomInterval()

def animateAllTrials(eid, baseDir, fps=24, maxTrials=None):
    """
    Animate all trials in a session with memory optimization and progress tracking.

    Args:
        eid (str): Experiment ID
        baseDir (str): Base directory for saving
        fps (int): Frame rate
        maxTrials (int, optional): Limit number of trials (for testing)

    Returns:
        bool: True if all trials succeeded, False otherwise
    """
    from tqdm import tqdm
    import gc
    import matplotlib.pyplot as plt

    # Load session once with VERBOSE output
    print("Loading session data...")

    dataLoader = IBLDataLoader(eid=eid, verbose=True)  # Enable verbose for troubleshooting
    print(f"DataLoader initialized for EID: {eid}")
    if not dataLoader.loadCompleteSession():
        print("Failed to load session")
        return False
    print(f"Session data loaded successfully - {len(dataLoader.trialData)} trials available")

    numTrials = len(dataLoader.trialData)
    if maxTrials:
        numTrials = min(numTrials, maxTrials)

    successCount = 0
    failedTrials = []

    # Create progress bar
    pbar = tqdm(range(numTrials), desc="Animating trials", unit="trial")

    for trialIdx in pbar:
        pbar.set_description(f"Processing trial {trialIdx+1}/{numTrials}")

        try:
            # Create renderer with reduced chunk size for memory efficiency
            renderer = IBLAnimationRenderer(mode='save', baseDir=baseDir, fps=fps,
                                           verbose=False, one=dataLoader.one, eid=dataLoader.eid,
                                           dataLoader=dataLoader)
            renderer.chunkSize = 50  # Reduce memory usage

            # Animate trial
            success = renderer.animateSpecificTrial(trialIdx)

            if success:
                successCount += 1
                pbar.set_postfix({"Success": f"{successCount}/{trialIdx+1}", "Failed": len(failedTrials)})
            else:
                failedTrials.append(trialIdx)
                pbar.set_postfix({"Success": f"{successCount}/{trialIdx+1}", "Failed": len(failedTrials)})

        except Exception as e:
            failedTrials.append(trialIdx)
            pbar.set_postfix({"Success": f"{successCount}/{trialIdx+1}", "Failed": len(failedTrials)})
            # Log error without interrupting progress bar
            tqdm.write(f"Error on trial {trialIdx}: {e}")

        finally:
            # Critical cleanup to prevent memory issues
            plt.close('all')
            if 'renderer' in locals():
                # Clear both camera caches
                for cache in renderer.videoFrameCaches.values():
                    cache.clear()
                del renderer
            gc.collect()

    pbar.close()

    # Final summary
    print(f"\nBatch processing completed:")
    print(f"  Successful: {successCount}/{numTrials} trials")
    print(f"  Failed: {len(failedTrials)} trials")
    if failedTrials:
        print(f"  Failed trial indices: {failedTrials}")

    return successCount == numTrials

# Quick batch processing functions
def quickBatchAnimate(eid, baseDir, maxTrials=None, fps=24):
    """
    Quick batch animation for testing (limits to first few trials).

    Args:
        eid (str): Experiment ID
        baseDir (str): Base directory for saving
        maxTrials (int): Maximum number of trials to process
        fps (int): Frame rate
    """
    print(f"Quick batch test: processing first {maxTrials} trials")
    return animateAllTrials(eid, baseDir, fps=fps, maxTrials=maxTrials)

# Test

In [ ]:
eid = '88d24c31-52e4-49cc-9f32-6adbeb9eba87'
BASE_DIR = "/content/drive/MyDrive/S25/Langone/Breathing/Videos"

In [ ]:
animateAllTrials(eid, BASE_DIR)

Loading session data...
Initializing IBL DataLoader for experiment: 88d24c31-52e4-49cc-9f32-6adbeb9eba87
DataLoader initialized for EID: 88d24c31-52e4-49cc-9f32-6adbeb9eba87
STARTING COMPLETE SESSION LOADING

Loading session data...
Loading session data...
2025-06-26 20:27:09 INFO     one.py:1475 Loading trials data


INFO:ibllib:Loading trials data


2025-06-26 20:27:12 INFO     one.py:1475 Loading wheel data


INFO:ibllib:Loading wheel data


2025-06-26 20:27:14 INFO     one.py:1475 Loading pose data


...

  Pose data in interval: 270 timepoints
    High quality features (>80%): 12
    Medium quality features (20-80%): 4
    Low quality features (<20%): 2
    Low quality features will rely on interpolation: ['pupil_bottom_r_x', 'pupil_bottom_r_y']
✓ Wheel data extracted: 4510 timepoints
  Time range: 145.634 - 150.143s
  Position range: 40.506313 - 43.209164
  Valid positions: 4510
Motion energy data type: <class 'pandas.core.frame.DataFrame'>
Motion energy DataFrame columns: ['times', 'whiskerMotionEnergy']
Using column 'whiskerMotionEnergy' for whisker motion energy
✓ Motion energy extracted: 270 timepoints
  Time range: 145.638 - 150.139s
  Data range: 0.855 - 13.935


Processing trial 19/547:   3%|▎         | 18/547 [3:10:52<80:55:58, 550.77s/trial, Success=18/18, Failed=0]

  Pose data in interval: 142 timepoints
    High quality features (>80%): 10
    Medium quality features (20-80%): 6
    Low quality features (<20%): 2
    Low quality features will rely on interpolation: ['pupil_bottom_r_x', 'pupil_bottom_r_y']
✓ Wheel data extracted: 2379 timepoints
  Time range: 150.616 - 152.994s
  Position range: 39.354294 - 40.742500
  Valid positions: 2379
Motion energy data type: <class 'pandas.core.frame.DataFrame'>
Motion energy DataFrame columns: ['times', 'whiskerMotionEnergy']
Using column 'whiskerMotionEnergy' for whisker motion energy
✓ Motion energy extracted: 142 timepoints
  Time range: 150.624 - 152.983s
  Data range: 1.062 - 14.707


Processing trial 20/547:   3%|▎         | 19/547 [3:15:54<69:50:34, 476.20s/trial, Success=19/19, Failed=0]

  Pose data in interval: 465 timepoints
    High quality features (>80%): 12
    Medium quality features (20-80%): 4
    Low quality features (<20%): 2
    Low quality features will rely on interpolation: ['pupil_bottom_r_x', 'pupil_bottom_r_y']
✓ Wheel data extracted: 7781 timepoints
  Time range: 153.365 - 161.145s
  Position range: 37.459881 - 40.595261
  Valid positions: 7781
Motion energy data type: <class 'pandas.core.frame.DataFrame'>
Motion energy DataFrame columns: ['times', 'whiskerMotionEnergy']
Using column 'whiskerMotionEnergy' for whisker motion energy
✓ Motion energy extracted: 465 timepoints
  Time range: 153.368 - 161.131s
  Data range: 0.990 - 15.668


# Good Result

In [ ]:
# Then animate inline
success = quickAnimate(dataLoader, mode='test', fps=30)

Output hidden; open in https://colab.research.google.com to view.